In [68]:
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


In [69]:
train_dir = 'data/train'
val_dir = 'data/test'

In [70]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [71]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [72]:

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [73]:
emotion_model = Sequential()

In [74]:
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

In [75]:
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

In [76]:
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
# emotion_model.load_weights('emotion_model.h5')

In [77]:
cv2.ocl.setUseOpenCL(False)

In [78]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [79]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=7178 // 64)
emotion_model.save_weights('emotion_model.h5')

C:\Users\Vignesh\AppData\Local\Temp\ipykernel_3828\3870905249.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


448/448 [==============================] - 478s 1s/step - loss: 1.8034 - accuracy: 0.2575 - val_loss: 1.7218 - val_accuracy: 0.3514


In [116]:
# start the webcam feed
cap = cv2.VideoCapture(1)
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier('opencv-master/data/haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = frame[y:y + h, x:x + w]
        temp1 = cv2.resize(roi_gray_frame, (48, 48))
        temp = np.expand_dims(cv2.resize(temp1,(48,48)), -1)
        temp.shape=(3,48,48,1)
        emotion_prediction = emotion_model.predict(temp)
        maxindex = int(np.argmax(emotion_prediction))
        print(emotion_dict[maxindex])
        #cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

1/1 [==============================] - 0s 57ms/step
Surprised
1/1 [==============================] - 0s 48ms/step
Surprised
1/1 [==============================] - 0s 39ms/step
Surprised
1/1 [==============================] - 0s 73ms/step
Surprised
1/1 [==============================] - 0s 71ms/step
Surprised
1/1 [==============================] - 0s 76ms/step
Surprised
1/1 [==============================] - 0s 46ms/step
Surprised
1/1 [==============================] - 0s 51ms/step
Surprised
1/1 [==============================] - 0s 55ms/step
Surprised
1/1 [==============================] - 0s 73ms/step
Surprised
1/1 [==============================] - 0s 64ms/step
Surprised
1/1 [==============================] - 0s 79ms/step
Surprised
1/1 [==============================] - 0s 70ms/step
Surprised
1/1 [==============================] - 0s 69ms/step
Surprised
1/1 [==============================] - 0s 62ms/step
Surprised
1/1 [==============================] - 0s 69ms/step
Surprised
1/1 [===

KeyboardInterrupt: 

In [117]:
cap.release()
cv2.destroyAllWindows()